## Introduction

Ecosystems around the world are now experiencing dramatic fluctuations in climate, and national parks are no exception. The purpose of this paper is to examine and describe the effects of this climate change on the national parks of southeast Utah; Arches National Park, Canyonlands National Park, Hovenweep National Monument, and Natural Bridges National Park. This analysis utilizes climate data to conduct past assessments and future projections to provide a means of evaluating the temporal dynamics of embodied concepts such as soil moisture, vegetation cover, temperature, and precipitation. The hope is to provide park managers and other interested stakeholders with information that may serve as a guide for implementing actions that in some way will reduce the adverse impact of climate change on park ecosystems.

## Data Overview

In this project, two datasets are used. First, there is the ‘NABR_historic.csv’ dataset, which comprises historical climate data. The dataset offers a starting point for understanding previous climate grounds and trends. The second dataset used by our project is the ‘nearterm_data_2020-2024.csv’ dataset, which hosts near-term climate forecasts obtained from multiple Representative Concentration Pathway scenarios. By integrating these two datasets, one can develop a comprehensive picture of climate trends and their potential implications for park ecosystems. It is possible to spot the trends by comparing former trends with new developments by using past data and future climate projections.(@source)

In [56]:
import pandas as pd
df = pd.read_csv('Data/NABR_historic.csv')
df.head()


,long,lat,year,TimePeriod,RCP,scenario,treecanopy,Ann_Herb,Bare,Herb,...,PPT_Annual,T_Winter,T_Summer,T_Annual,Tmax_Summer,Tmin_Winter,VWC_Winter_whole,VWC_Spring_whole,VWC_Summer_whole,VWC_Fall_whole
0,-110.0472,37.60413,2021,NT,4.5,sc22,0,0,84,5,...,6.37,1.630333,24.50402,24.50402,36.89,NaN,NaN,NaN,NaN,NaN
1,-110.0472,37.60413,2021,NT,4.5,sc22,0,0,84,5,...,NaN,NaN,NaN,NaN,NaN,-12.77,0.114652,0.078764,0.043514,0.051281
2,-110.0472,37.60413,2021,NT,4.5,sc23,0,0,84,5,...,3.09,1.389056,24.11043,24.11043,37.95,NaN,NaN,NaN,NaN,NaN
3,-110.0472,37.60413,2021,NT,4.5,sc23,0,0,84,5,...,NaN,NaN,NaN,NaN,NaN,-18.96,0.130221,0.096412,0.041232,0.092241
4,-110.0472,37.60413,2021,NT,4.5,sc24,0,0,84,5,...,6.87,-0.334389,25.54266,10.31321,37.74,NaN,NaN,NaN,NaN,NaN


In [2]:
df2 = pd.read_csv('Data/nearterm_data_2020-2024.csv')
df2.head()

merged_df = pd.concat([df, df2], ignore_index=True)

merged_df = merged_df.loc[:, ~merged_df.columns.duplicated()]

merged_df.dropna(how='all', inplace=True)


,long,lat,year,TimePeriod,RCP,scenario,treecanopy,Ann_Herb,Bare,Herb,...,PPT_Annual,T_Winter,T_Summer,T_Annual,Tmax_Summer,Tmin_Winter,VWC_Winter_whole,VWC_Spring_whole,VWC_Summer_whole,VWC_Fall_whole
0,-110.0472,37.60413,2021,NT,4.5,sc22,0,0,84,5,...,6.37,1.630333,24.50402,24.50402,36.89,NaN,NaN,NaN,NaN,NaN
1,-110.0472,37.60413,2021,NT,4.5,sc22,0,0,84,5,...,NaN,NaN,NaN,NaN,NaN,-12.77,0.114652,0.078764,0.043514,0.051281
2,-110.0472,37.60413,2021,NT,4.5,sc23,0,0,84,5,...,3.09,1.389056,24.11043,24.11043,37.95,NaN,NaN,NaN,NaN,NaN
3,-110.0472,37.60413,2021,NT,4.5,sc23,0,0,84,5,...,NaN,NaN,NaN,NaN,NaN,-18.96,0.130221,0.096412,0.041232,0.092241
4,-110.0472,37.60413,2021,NT,4.5,sc24,0,0,84,5,...,6.87,-0.334389,25.54266,10.31321,37.74,NaN,NaN,NaN,NaN,NaN


## Data filtering and cleaning process

To begin data cleaning, we downloaded two datasets: nearterm_data_2020-2024.csv, which contained near-term climate projections, and NABR_historic.csv, which contained historical climate data. These datasets were combined into a single DataFrame, removing all rows with missing values ​​and ensuring that there were no duplicate fields. This first step removed duplicates and completely empty records by merging relevant data.


Next we determined which columns are appropriate for each aggregation technique. For the columns representing vegetation (Canopy, Ann_Herb, Bare, Herb, Litter, Arbusb), we determined the highest values ​​for each group. We used a custom function to correctly remove nulls and NaNs from averages of other columns. The DataFrame was then grouped using the appropriate aggregation functions for long, lat, year, TimePeriod, RCP and scenario.

Finally, we cleaned the dataset by removing the DrySoilDays_Summer_whole, PPT_Annual, and T_Annual columns that had large missing values ​​or were not needed for further analysis. Possible missing values ​​ExtremeShortTermDryStress_Summer_whole and NonDrySWA_Summer_whole were filled with the mean of the corresponding column. This comprehensive method provided a clean, consistent and complete data set that allowed for an in-depth study of climate impacts on the Southeastern Utah Group of National Parks.

In [4]:
max_columns = ['treecanopy', 'Ann_Herb', 'Bare', 'Herb', 'Litter', 'Shrub']

group_by_columns = ['long', 'lat', 'year', 'TimePeriod', 'RCP', 'scenario']
avg_columns = [col for col in merged_df.columns if col not in max_columns + group_by_columns]

def custom_avg(series):
    return series[series != 0].mean()

aggregated_df = merged_df.groupby(group_by_columns).agg(
    {**{col: 'max' for col in max_columns},
     **{col: custom_avg for col in avg_columns}}
).reset_index()

aggregated_df.drop(columns=['DrySoilDays_Summer_whole', 'PPT_Annual', 'T_Annual'], inplace=True)

aggregated_df['ExtremeShortTermDryStress_Summer_whole'].fillna(aggregated_df['ExtremeShortTermDryStress_Summer_whole'].mean(), inplace=True)
aggregated_df['NonDrySWA_Summer_whole'].fillna(aggregated_df['NonDrySWA_Summer_whole'].mean(), inplace=True)
aggregated_df


,long,lat,year,TimePeriod,RCP,scenario,treecanopy,Ann_Herb,Bare,Herb,...,PPT_Winter,PPT_Summer,T_Winter,T_Summer,Tmax_Summer,Tmin_Winter,VWC_Winter_whole,VWC_Spring_whole,VWC_Summer_whole,VWC_Fall_whole
0,-110.0472,37.60413,1980,Hist,historical,sc1,0,0,84,5,...,13.79,2.69,0.964835,23.15924,37.05,-12.45,0.113447,0.096831,0.041876,0.052298
1,-110.0472,37.60413,1981,Hist,historical,sc1,0,0,84,5,...,2.25,9.39,3.334444,23.27065,37.55,-9.35,0.049382,0.060727,0.042639,0.093671
2,-110.0472,37.60413,1982,Hist,historical,sc1,0,0,84,5,...,4.12,9.50,-0.015556,22.05707,36.65,-16.55,0.109234,0.074817,0.045610,0.067789
3,-110.0472,37.60413,1983,Hist,historical,sc1,0,0,84,5,...,7.09,10.22,0.409444,21.32826,34.55,-15.05,0.120418,0.102542,0.044141,0.074802
4,-110.0472,37.60413,1984,Hist,historical,sc1,0,0,84,5,...,4.77,9.49,-1.047253,21.95978,35.35,-18.45,0.120209,0.077842,0.043179,0.070366
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
22482,-109.9659,37.62525,2024,NT,8.5,sc57,18,0,37,18,...,6.25,8.49,3.372912,23.75413,38.02,-12.16,0.142255,0.173792,0.091433,0.106907
22483,-109.9659,37.62525,2024,NT,8.5,sc58,18,0,37,18,...,7.49,4.13,2.162582,26.91342,46.77,-12.16,0.166254,0.158146,0.086355,0.095905
22484,-109.9659,37.62525,2024,NT,8.5,sc59,18,0,37,18,...,16.28,9.70,1.209121,23.48859,37.08,-10.38,0.151342,0.175814,0.091992,0.109333
22485,-109.9659,37.62525,2024,NT,8.5,sc60,18,0,37,18,...,2.00,8.15,1.549945,23.86120,37.06,-10.61,0.122249,0.104884,0.087115,0.118908


## Temperature Overview

In [22]:
import plotly.express as px
import plotly.graph_objects as go

filtered_df = aggregated_df
avg_temp = filtered_df.groupby(['year', 'RCP'])[['T_Summer', 'T_Winter']].mean().reset_index()

fig_summer_temp = px.line(
    avg_temp,
    x='year',
    y='T_Summer',
    color='RCP',
    labels={'T_Summer': 'Summer Temperature (°C)', 'year': 'Year', 'RCP': 'Scenario'},
    title='Historical vs. Projected Summer Temperature Trends'
)
fig_summer_temp.update_layout(legend_title_text='RCP')

fig_winter_temp = px.line(
    avg_temp,
    x='year',
    y='T_Winter',
    color='RCP',
    labels={'T_Winter': 'Winter Temperature (°C)', 'year': 'Year', 'RCP': 'Scenario'},
    title='Historical vs. Projected Winter Temperature Trends'
)
fig_winter_temp.update_layout(legend_title_text='RCP')
fig_winter_temp.write_html("fig_winter_temp.html")
fig_summer_temp.write_html("fig_summer_temp.html")



<iframe src="./fig_winter_temp.html" width="120%" height="600px"></iframe>
<iframe src="./fig_summer_temp.html" width="120%" height="600px"></iframe>

In this chart, you can see the inverse relationships with the extreme line charts of the average temperature in summer and winter for the RCP 8.5 and 4.5. From the linear trend over time in interactive line graphs of average temperatures in summer and winter for RCP Sequences 8.5 and 4.5, showing the historic and near-term. The linear trend shows that the RCP 8.5 sequence seems to be an ever more short-term increases in temperature, without power measures to reduce emissions. From our analysis, the parks are likely to suffer from the hot summer and winters which would significantly impact ecosystems. The linear increase in average temperature over time is further supported by the interactive line graphs for summer and winter temperature, more particularly on the RCP 8.5 sequence. These will have a drastic impact on the volume of precipitation required for the ecological balance of the national parks located in southeastern Utah.

## Seasonal Precipitation Analysis

In [25]:
filtered_df = aggregated_df
avg_precipitation = filtered_df.groupby(['year', 'RCP'])[['PPT_Summer', 'PPT_Winter']].mean().reset_index()

fig_summer_ppt = px.bar(
    avg_precipitation,
    x='year',
    y='PPT_Summer',
    color='RCP',
    barmode='group',
    labels={'PPT_Summer': 'Average Summer Precipitation (cm)', 'year': 'Year', 'RCP': 'Scenario'},
    title='Historical vs. Projected Summer Precipitation'
)
fig_summer_ppt.update_layout(legend_title_text='RCP')

fig_winter_ppt = px.bar(
    avg_precipitation,
    x='year',
    y='PPT_Winter',
    color='RCP',
    barmode='group',
    labels={'PPT_Winter': 'Average Winter Precipitation (cm)', 'year': 'Year', 'RCP': 'Scenario'},
    title='Historical vs. Projected Winter Precipitation'
)
fig_winter_ppt.update_layout(legend_title_text='RCP')

fig_summer_ppt.write_html("fig_summer_ppt.html")
fig_winter_ppt.write_html("fig_winter_ppt.html")

<iframe src="./fig_winter_ppt.html" width="120%" height="600px"></iframe>
<iframe src="./fig_summer_ppt.html" width="120%" height="600px"></iframe>

We further look at breaking down these trends by the use of seasonal precipitation. Bar charts show signs of precipitation change with respect to time and reveal past discrepancies and the RCP 4.5 and 8.5 projections in summer and winter rain performance. Since the changes in precipitation have the ability to change the overall distress of the ecosystem, the vegetation growth, and the availability of water, it is important to investigate these changes. Our analysis on changes in temperature and precipitation holds a full view of potential impacts across our assigned areas of conservation: historic and future-projected summer and winter precipitation now shed light on just what those impacts could be.

## Soil Moisture Dynamics

In [55]:
avg_soil_moisture = aggregated_df.groupby(['year', 'RCP'])[['VWC_Winter_whole', 'VWC_Spring_whole', 'VWC_Summer_whole', 'VWC_Fall_whole']].mean().reset_index()

fig_soil_moisture_winter = px.line(
    avg_soil_moisture,
    x='year',
    y='VWC_Winter_whole',
    color='RCP',
    labels={'VWC_Winter_whole': 'Winter Volumetric Water Content (m³/m³)', 'year': 'Year', 'RCP': 'Scenario'},
    title='Historical vs. Projected Winter Soil Moisture'
)
fig_soil_moisture_winter.update_layout(legend_title_text='RCP')

fig_soil_moisture_spring = px.line(
    avg_soil_moisture,
    x='year',
    y='VWC_Spring_whole',
    color='RCP',
    labels={'VWC_Spring_whole': 'Spring Volumetric Water Content (m³/m³)', 'year': 'Year', 'RCP': 'Scenario'},
    title='Historical vs. Projected Spring Soil Moisture'
)
fig_soil_moisture_spring.update_layout(legend_title_text='RCP')

fig_soil_moisture_summer = px.line(
    avg_soil_moisture,
    x='year',
    y='VWC_Summer_whole',
    color='RCP',
    labels={'VWC_Summer_whole': 'Summer Volumetric Water Content (m³/m³)', 'year': 'Year', 'RCP': 'Scenario'},
    title='Historical vs. Projected Summer Soil Moisture'
)
fig_soil_moisture_summer.update_layout(legend_title_text='RCP')

fig_soil_moisture_fall = px.line(
    avg_soil_moisture,
    x='year',
    y='VWC_Fall_whole',
    color='RCP',
    labels={'VWC_Fall_whole': 'Fall Volumetric Water Content (m³/m³)', 'year': 'Year', 'RCP': 'Scenario'},
    title='Historical vs. Projected Fall Soil Moisture'
)
fig_soil_moisture_fall.update_layout(legend_title_text='RCP')

fig_soil_moisture_winter.write_html("fig_soil_moisture_winter.html")
fig_soil_moisture_spring.write_html("fig_soil_moisture_spring.html")
fig_soil_moisture_summer.write_html("fig_soil_moisture_summer.html")
fig_soil_moisture_fall.write_html("fig_soil_moisture_fall.html")



<iframe src="./fig_soil_moisture_winter.html" width="120%" height="600px"></iframe>
<iframe src="./fig_soil_moisture_spring.html" width="120%" height="600px"></iframe>
<iframe src="./fig_soil_moisture_summer.html" width="120%" height="600px"></iframe>
<iframe src="./fig_soil_moisture_fall.html" width="120%" height="600px"></iframe>

Following the precipitation patterns, we analyzed the soil moisture trends, which are the basis of ensuring the health of park ecosystems. Soil moisture is one of the key factors in the sustainability of ecosystems, plant health, and water availability. A comparison of VWC in the different seasons—winter, spring, summer, and fall—can infer how these changes may affect the health of plants and park soils. Evidently, from the interactive line graphs, it shows that the soil moistures have seriously changed over the past decades. However, the future values for the RCP 8.5 sequence are showing the trend of an increase, which is significant for summer. It is an indication that many national parks may have suffered severe droughts where plants have died off and species composition in the park is further altered. This overview of the four seasons and potential impacts throws light on the policy needs for the future to mitigate the impacts of climate change on soil moisture.

## Extreme Weather Events analysis

In [54]:
avg_extreme_weather = aggregated_df.groupby(['year', 'RCP'])[['FrostDays_Winter', 'ExtremeShortTermDryStress_Summer_whole']].mean().reset_index()
fig_frost_days = px.bar(
    avg_extreme_weather,
    x='year',
    y='FrostDays_Winter',
    color='RCP',
    barmode='group',
    labels={'FrostDays_Winter': 'Number of Frost Days', 'year': 'Year', 'RCP': 'Scenario'},
    title='Historical vs. Projected Winter Frost Days'
)
fig_frost_days.update_layout(legend_title_text='RCP')

fig_dry_stress = px.bar(
    avg_extreme_weather,
    x='year',
    y='ExtremeShortTermDryStress_Summer_whole',
    color='RCP',
    barmode='group',
    labels={'ExtremeShortTermDryStress_Summer_whole': 'Extreme Short-Term Dry Stress (°C)', 'year': 'Year', 'RCP': 'Scenario'},
    title='Historical vs. Projected Extreme Short-Term Dry Stress'
)
fig_dry_stress.update_layout(legend_title_text='RCP')

fig_frost_days.write_html("fig_frost_days.html")
fig_dry_stress.write_html("fig_dry_stress.html")


<iframe src="./fig_frost_days.html" width="120%" height="600px"></iframe>
<iframe src="./fig_dry_stress.html" width="120%" height="600px"></iframe>

The last segment of our research is devoted to extreme weather, especially severe drought stress in summer and cold days in winter. These movements might severely affect park ecosystems, degrading biodiversity, water resources, and impossible plant survival. By comparing the RCP 4.5 and RCP 8.5 future expected occurrence with existing historical data, people may project future events. Frosting days have varied over time, as shown by the bar graphs, but are expected to continue under the RCP 8.5 series. Conversely, more intense temporary drought stress is predicted during the summer, resulting in warmer and dehydrated climate. Hence, parks are expected to endure more severe, recurring droughts, which may kill off plants and change ecosystem composition. Understanding these trends is essential for creating strategies that could increase conservation areas’ natural value.

## Climate Impact Features for 2024

In [53]:
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go

df_2024 = aggregated_df[aggregated_df['year'] == 2024]

features = ['VWC_Winter_whole', 'VWC_Spring_whole', 'VWC_Summer_whole', 'VWC_Fall_whole', 
            'FrostDays_Winter', 'ExtremeShortTermDryStress_Summer_whole']

initial_feature = features[0]

fig = px.scatter_mapbox(
    df_2024,
    lat="lat",
    lon="long",
    hover_name="year",
    hover_data={initial_feature: True, 'RCP': True},
    color=initial_feature,
    color_continuous_scale=[[0, 'white'], [1, 'red']],
    size_max=15,
    zoom = 11,
    mapbox_style="carto-positron"
)

buttons = []
for feature in features:
    buttons.append(
        dict(
            method='update',
            label=feature,
            args=[
                {
                    'marker.color': [df_2024[feature]],
                    'hovertemplate': f'<b>2024</b><br>lat=%{{lat}}<br>long=%{{lon}}<br>RCP=%{{customdata[1]}}<br>{feature}=%{{marker.color}}<extra></extra>',
                }
            ]
        )
    )

fig.update_layout(
    title="Climate Impact Features for 2024",
    coloraxis_colorbar=dict(
        title="Feature Value",
    ),
    updatemenus=[dict(
        buttons=buttons,
        direction='down',
        showactive=True,
        x=0.4,
        xanchor='left',
        y=1.2,
        yanchor='top'
    )]
)

fig.write_html("climate_impact_features_map.html")






<iframe src="./climate_impact_features_map.html" width="120%" height="600px"></iframe>

Finally, we compile an interactive map for year 2024 to predict the state and impacts of climate change for the years was selected. The interactive map landscapes various park locations, informing the user about the location’s extremes and soil moistures. Another drop-down selection enables the user to identify a function that tracks spatial trends, assisting them in selecting park extremes and vulnerable locations that require special care and attention. The picture is critical for future analysis and planning because it indicates areas deserving more focus as a result of rapid going weather.

## Conclusion

Borrowing this comprehensive knowledge from potential organizational impacts, in conclusion, the Southeastern Utah Group of National Parks is a vital trend published. Temperature, precipitation, soil moistures, and extenuating occurrences are all significant factors to consider. By studying historic trends and future predictions under various RCP scenarios, one better understands relevant trends and variations. Our analysis indicates a semi-lineal increase in temperature will lead to warmer summers and mild winters. Such changes will be followed shortly after by precipitation, drastically lowering soil moisture throughout the summer. The drought stress indicated will be enough to modify plant composition and vegetation formation. Separately, an increase in extenuating phenomena including short-term drought pulls and reduced frostings proposes that environmental components pattern and respond to increased deviations. Various foundation changes are required in the form of alterable administration modules.